<a href="https://colab.research.google.com/github/sunil-karki/Google3QLA-col/blob/master/Google3QLA_COLAB_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import files
uploaded = files.upload()

Saving sample_submission.csv to sample_submission.csv
Saving test.csv to test.csv
Saving train.csv to train (1).csv


In [0]:
import io
import pandas as pd
X_train = pd.read_csv(io.BytesIO(uploaded['train.csv']))
# Dataset is now stored in a Pandas Dataframe

#X_train = pd.read_csv("../input/google-quest-challenge/train.csv")
print( len(X_train.columns), ' ', len(X_train))

41   6079


In [0]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [0]:
!pip install kaggle

In [3]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"sunilkarki520","key":"8368d7fdc07a580b54aeb9a6df65cda4"}'}

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle competitions download -c google-quest-challenge 

  0% 0.00/954k [00:00<?, ?B/s]
100% 954k/954k [00:00<00:00, 60.6MB/s]
100% 4.50M/4.50M [00:00<00:00, 20.2MB/s]

  0% 0.00/115k [00:00<?, ?B/s]
100% 115k/115k [00:00<00:00, 119MB/s]


In [6]:
!ls

kaggle.json  sample_data  sample_submission.csv  test.csv  train.csv.zip


In [0]:
import pandas as pd
#df = pd.read_csv('filename.zip', compression='zip', header=0, sep=',', quotechar='"')
X_train = pd.read_csv('train.csv.zip', compression='zip') 

#X_train[['qa_id','question_title','question_body','question_user_name','question_user_page','answer','answer_user_name','answer_user_page','url',
#        'category','host']]

In [0]:
test_file = pd.read_csv('test.csv')

In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

In [9]:
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


In [0]:
#Imports
from keras.models import Sequential, Model
from keras.layers.embeddings import Embedding
from keras.layers import Input, Activation, Dense, Permute, Dropout, add, dot, concatenate, LSTM

from keras import optimizers

from sklearn.model_selection import train_test_split

In [107]:
X_train[['qa_id','question_title','question_body','question_user_name','question_user_page','answer','answer_user_name','answer_user_page','url',
        'category','host']]

,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,0,What am I losing when using extension tubes in...,After playing around with macro photography on...,ysap,https://photo.stackexchange.com/users/1024,"I just got extension tubes, so here's the skin...",rfusca,https://photo.stackexchange.com/users/1917,http://photo.stackexchange.com/questions/9169/...,LIFE_ARTS,photo.stackexchange.com
1,1,What is the distinction between a city and a s...,I am trying to understand what kinds of places...,russellpierce,https://rpg.stackexchange.com/users/8774,It might be helpful to look into the definitio...,Erik Schmidt,https://rpg.stackexchange.com/users/1871,http://rpg.stackexchange.com/questions/47820/w...,CULTURE,rpg.stackexchange.com
2,2,Maximum protusion length for through-hole comp...,I'm working on a PCB that has through-hole com...,Joe Baker,https://electronics.stackexchange.com/users/10157,Do you even need grooves? We make several pro...,Dwayne Reid,https://electronics.stackexchange.com/users/64754,http://electronics.stackexchange.com/questions...,SCIENCE,electronics.stackexchange.com
3,3,Can an affidavit be used in Beit Din?,"An affidavit, from what i understand, is basic...",Scimonster,https://judaism.stackexchange.com/users/5151,"Sending an ""affidavit"" it is a dispute between...",Y e z,https://judaism.stackexchange.com/users/4794,http://judaism.stackexchange.com/questions/551...,CULTURE,judaism.stackexchange.com
4,5,How do you make a binary image in Photoshop?,I am trying to make a binary image. I want mor...,leigero,https://graphicdesign.stackexchange.com/users/...,Check out Image Trace in Adobe Illustrator. \n...,q2ra,https://graphicdesign.stackexchange.com/users/...,http://graphicdesign.stackexchange.com/questio...,LIFE_ARTS,graphicdesign.stackexchange.com
...,...,...,...,...,...,...,...,...,...,...,...
6074,9642,Using a ski helmet for winter biking,I am curious if anyone uses a skiing helmet fo...,sixtyfootersdude,https://bicycles.stackexchange.com/users/134,If you're thinking about wearing a ski helmet ...,Matt Leo,https://bicycles.stackexchange.com/users/3340,http://bicycles.stackexchange.com/questions/99...,CULTURE,bicycles.stackexchange.com
6075,9643,Adjustment to road bike brakes for high grade ...,I have a road bike with a front brake that wea...,ash,https://bicycles.stackexchange.com/users/14519,\nYou can replace the pads (as stated elsewher...,Daniel R Hicks,https://bicycles.stackexchange.com/users/1584,http://bicycles.stackexchange.com/questions/25...,CULTURE,bicycles.stackexchange.com
6076,9645,Suppress 'file truncated' messages when using ...,I'm tailing a log file using tail -f messages....,Maneating Koala,https://unix.stackexchange.com/users/60445,Maybe help if can be fixes origin of this erro...,BG Bruno,https://unix.stackexchange.com/users/68208,http://unix.stackexchange.com/questions/169054...,TECHNOLOGY,unix.stackexchange.com
6077,9646,When should a supervisor be a co-author?,What are people's views on this? To be specif...,MrB,https://mathoverflow.net/users/2189,"As a non-mathematician, I am somewhat mystifie...",angela,https://mathoverflow.net/users/4267,http://mathoverflow.net/questions/57337,SCIENCE,mathoverflow.net


In [108]:
X_train[['qa_id','question_title','question_body','answer','category']]

,qa_id,question_title,question_body,answer,category
0,0,What am I losing when using extension tubes in...,After playing around with macro photography on...,"I just got extension tubes, so here's the skin...",LIFE_ARTS
1,1,What is the distinction between a city and a s...,I am trying to understand what kinds of places...,It might be helpful to look into the definitio...,CULTURE
2,2,Maximum protusion length for through-hole comp...,I'm working on a PCB that has through-hole com...,Do you even need grooves? We make several pro...,SCIENCE
3,3,Can an affidavit be used in Beit Din?,"An affidavit, from what i understand, is basic...","Sending an ""affidavit"" it is a dispute between...",CULTURE
4,5,How do you make a binary image in Photoshop?,I am trying to make a binary image. I want mor...,Check out Image Trace in Adobe Illustrator. \n...,LIFE_ARTS
...,...,...,...,...,...
6074,9642,Using a ski helmet for winter biking,I am curious if anyone uses a skiing helmet fo...,If you're thinking about wearing a ski helmet ...,CULTURE
6075,9643,Adjustment to road bike brakes for high grade ...,I have a road bike with a front brake that wea...,\nYou can replace the pads (as stated elsewher...,CULTURE
6076,9645,Suppress 'file truncated' messages when using ...,I'm tailing a log file using tail -f messages....,Maybe help if can be fixes origin of this erro...,TECHNOLOGY
6077,9646,When should a supervisor be a co-author?,What are people's views on this? To be specif...,"As a non-mathematician, I am somewhat mystifie...",SCIENCE


In [109]:
test_file

,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com
...,...,...,...,...,...,...,...,...,...,...,...
471,9569,change gb4e enumeration in only one chapter,I am using gb4e for glossing examples in my th...,ToddAO,https://tex.stackexchange.com/users/28332,You can simply reset the exx counter just afte...,karlkoeller,https://tex.stackexchange.com/users/27635,http://tex.stackexchange.com/questions/165120/...,TECHNOLOGY,tex.stackexchange.com
472,9590,All anagrams in a File,Source : Microsoft Interview Question\n\nWe ar...,Spandan,https://stackoverflow.com/users/2426101,Slightly different approach from the one above...,Algorithmatic,https://stackoverflow.com/users/1122229,http://stackoverflow.com/questions/16872513/al...,STACKOVERFLOW,stackoverflow.com
473,9597,SQL Server: Time Series Table Design,I am creating a time series database that foll...,Jay Michael,https://dba.stackexchange.com/users/47001,A foreign key can reference a unique constrain...,mustaccio,https://dba.stackexchange.com/users/23721,http://dba.stackexchange.com/questions/105287/...,TECHNOLOGY,dba.stackexchange.com
474,9623,Is a countered flashback spell exiled?,If I flashback Past in Flames and it is counte...,BolasStone,https://boardgames.stackexchange.com/users/9981,If you counter a spell cast for its Flashback ...,murgatroid99,https://boardgames.stackexchange.com/users/2537,http://boardgames.stackexchange.com/questions/...,CULTURE,boardgames.stackexchange.com


In [0]:
#X_train['question_body'][0]
##  https://www.figure-eight.com/data-for-everyone/

In [0]:
# 'question_title','question_body','category' to be added together here
X_train['question_body'] = X_train['question_body'] + ' ' + X_train['question_title'] + ' ' + X_train['category']
del X_train['question_title']
del X_train['category']
#X_train['category']

In [112]:
X_train.columns

Index(['qa_id', 'question_body', 'question_user_name', 'question_user_page',
       'answer', 'answer_user_name', 'answer_user_page', 'url', 'host',
       'question_asker_intent_understanding', 'question_body_critical',
       'question_conversational', 'question_expect_short_answer',
       'question_fact_seeking', 'question_has_commonly_accepted_answer',
       'question_interestingness_others', 'question_interestingness_self',
       'question_multi_intent', 'question_not_really_a_question',
       'question_opinion_seeking', 'question_type_choice',
       'question_type_compare', 'question_type_consequence',
       'question_type_definition', 'question_type_entity',
       'question_type_instructions', 'question_type_procedure',
       'question_type_reason_explanation', 'question_type_spelling',
       'question_well_written', 'answer_helpful',
       'answer_level_of_information', 'answer_plausible', 'answer_relevance',
       'answer_satisfaction', 'answer_type_instructions',
 

In [113]:
import nltk
#Resource stopwords and wordnet not found
#needed to do this below in colab
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [0]:
#import nltk
import re

stopword = nltk.corpus.stopwords.words('english')

lm = nltk.WordNetLemmatizer()
st = nltk.PorterStemmer()

class NLP_preprocess:
    def tokenize(self, text):
        tokens = re.split('\W+', text)
        return tokens

    def remove_stopwords(self, tokenized_text):
        text = [word for word in tokenized_text if word not in stopword]
        return text
    
    def lemmatizing(self, text):
        text = [lm.lemmatize(word) for word in text]
        return text
    
    def stemming(self, text):
        text = [st.stem(word) for word in text]
        return text

    def process_column(self, col):
        col = col.apply(lambda x: self.tokenize(x.lower()))
        col = col.apply(lambda x: self.remove_stopwords(x))
        col = col.apply(lambda x: self.lemmatizing(x))
        return col


In [0]:
nlp = NLP_preprocess()

#Contains column which are text. Needs to be converted to Token
list_col = ['answer', 'question_body']

for col in list_col:
    X_train[col] = nlp.process_column(X_train[col])

In [0]:
class NLP_columns:
    def join_col(self, list_col):
        new_col = X_train[list_col[0]]
    
        for col in list_col[1:]:
            new_col += X_train[col]
        
        return new_col

    #Removing unnecessary elements from the processes joined Tokenized column
    def remove_values_from_list(self, the_list, val):
        return [value for value in the_list if value != val]
    
    def remove_values_from_column(self, col, val):
        return col.apply(lambda row: self.remove_values_from_list(row, val))


In [117]:
'''
def process_vocab():
    vocab = set()
    for a in X_train['answer']:
        vocab = vocab.union(set(a))

    for a in X_train['question_body']:
        vocab = vocab.union(set(a))
    
    vocab_len = len(vocab)
    #print(vocab_len)      #49474
    return vocab_len
    
vocab_len = process_vocab()
print(vocab_len)
'''

"\ndef process_vocab():\n    vocab = set()\n    for a in X_train['answer']:\n        vocab = vocab.union(set(a))\n\n    for a in X_train['question_body']:\n        vocab = vocab.union(set(a))\n    \n    vocab_len = len(vocab)\n    #print(vocab_len)      #49474\n    return vocab_len\n    \nvocab_len = process_vocab()\nprint(vocab_len)\n"

In [0]:
#print(vocab)
ylabels = ['question_asker_intent_understanding','question_body_critical','question_conversational','question_expect_short_answer',
'question_fact_seeking','question_has_commonly_accepted_answer','question_interestingness_others','question_interestingness_self',
'question_multi_intent','question_not_really_a_question','question_opinion_seeking','question_type_choice','question_type_compare',
'question_type_consequence','question_type_definition','question_type_entity','question_type_instructions','question_type_procedure',
'question_type_reason_explanation','question_type_spelling','question_well_written','answer_helpful','answer_level_of_information',
'answer_plausible','answer_relevance','answer_satisfaction','answer_type_instructions','answer_type_procedure','answer_type_reason_explanation',
'answer_well_written']

y = X_train[ylabels].astype(np.int64)



In [0]:
#Delete unwanted columns from X_train
#del X_train['question_asker_intent_understanding']
#X_train.drop(['question_asker_intent_understanding', 'question_body_critical'], axis=1, inplace=True)
X_train.drop(ylabels, axis=1, inplace=True)

In [120]:
X_train.columns     #Will need only  'answer' 'question_body'

Index(['qa_id', 'question_body', 'question_user_name', 'question_user_page',
       'answer', 'answer_user_name', 'answer_user_page', 'url', 'host'],
      dtype='object')

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y, test_size=0.33, random_state=np.random, shuffle=True)

In [122]:
print(X_test.shape)

(2007, 9)


In [123]:
'''
from sklearn.model_selection import train_test_split
X, y = np.arange(20).reshape((10, 2)), range(10)
print(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=np.random, shuffle=True)
print(X_train)
print(y_train)
'''

'\nfrom sklearn.model_selection import train_test_split\nX, y = np.arange(20).reshape((10, 2)), range(10)\nprint(X)\n\nX_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=np.random, shuffle=True)\nprint(X_train)\nprint(y_train)\n'

In [124]:
'''
class NLP_before_model:
    tokenizer = 0
    max_question_body_len = 0
    max_answers_seq_len = 0
    train_question_body = []
    train_answers = []
    
    x_answer = 0
    x_question = 0
    
    vocab = set()
    vocab_len = 0
    
    def __init__(self, x_answer, x_question):
        self.x_answer = x_answer
        self.x_question = x_question
        
        self.process_vocab()
        self.tokenizer = Tokenizer(filters = [])
        self.tokenizer.fit_on_texts(self.vocab)
        
        #self.process_for_model()
        
    def process_vocab(self):
        #vocab = set()
        for a in self.x_answer:
            self.vocab = self.vocab.union(set(a))

        for a in self.x_question:
            self.vocab = self.vocab.union(set(a))

        self.vocab_len = len(self.vocab)
        #print(vocab_len)      #49474
        #return self.vocab_len
        
    def process_for_model(self):
        for a in self.x_answer:
            self.train_answers.append(a)

        for a in self.x_question:
            self.train_question_body.append(a)
            
        a, b = self.process_vectorize(self.train_answers, self.train_question_body)
        return a, b
                    
    def process_vectorize(self, train_answers, train_question_body):
        
        train_answers_seq = self.tokenizer.texts_to_sequences(train_answers)
        train_question_body_seq = self.tokenizer.texts_to_sequences(train_question_body)

        self.max_question_body_len = max([len(data) for data in train_question_body_seq])        
        self.max_answers_seq_len = max([len(data) for data in train_answers_seq])
        
        input_answer = pad_sequences(train_answers_seq, maxlen = self.max_answers_seq_len)
        input_question = pad_sequences(train_question_body_seq, maxlen = self.max_question_body_len)

        return input_answer, input_question

'''

'\nclass NLP_before_model:\n    tokenizer = 0\n    max_question_body_len = 0\n    max_answers_seq_len = 0\n    train_question_body = []\n    train_answers = []\n    \n    x_answer = 0\n    x_question = 0\n    \n    vocab = set()\n    vocab_len = 0\n    \n    def __init__(self, x_answer, x_question):\n        self.x_answer = x_answer\n        self.x_question = x_question\n        \n        self.process_vocab()\n        self.tokenizer = Tokenizer(filters = [])\n        self.tokenizer.fit_on_texts(self.vocab)\n        \n        #self.process_for_model()\n        \n    def process_vocab(self):\n        #vocab = set()\n        for a in self.x_answer:\n            self.vocab = self.vocab.union(set(a))\n\n        for a in self.x_question:\n            self.vocab = self.vocab.union(set(a))\n\n        self.vocab_len = len(self.vocab)\n        #print(vocab_len)      #49474\n        #return self.vocab_len\n        \n    def process_for_model(self):\n        for a in self.x_answer:\n            se

In [0]:
class NLP_before_model:
    tokenizer = 0
    max_question_body_len = 0
    max_answers_seq_len = 0
    train_question_body = []
    train_answers = []

    train_answers_seq = 0
    train_question_body_seq = 0
    
    x_answer = 0
    x_question = 0
    
    vocab = set()
    vocab_len = 0
    
    def __init__(self, x_answer, x_question):
        self.x_answer = x_answer
        self.x_question = x_question
        
        self.process_vocab()
        self.tokenizer = Tokenizer(filters = [])
        self.tokenizer.fit_on_texts(self.vocab)
        
        #self.process_for_model()
        
    def process_vocab(self):
        #vocab = set()
        for a in self.x_answer:
            self.vocab = self.vocab.union(set(a))

        for a in self.x_question:
            self.vocab = self.vocab.union(set(a))

        self.vocab_len = len(self.vocab)
        #print(vocab_len)      #49474
        #return self.vocab_len

        self.max_question_len = max([len(data) for data in self.x_question])  #all_data = train + test
        self.max_story_len =    max([len(data) for data in self.x_answer])

    def tokenize_wordindex(self):
        print(self.tokenizer.word_index)


    def process_for_model(self):
        for a in self.x_answer:
            self.train_answers.append(a)

        for a in self.x_question:
            self.train_question_body.append(a)


    def vectorize_stories(self, data_answer, data_question):
        word_index = self.tokenizer.word_index
        max_story_len = self.max_story_len
        max_question_len = self.max_question_len
        #vectorized stories:
        X = []
        #vectorized questions:
        Xq = []
        #vectorized answers:
        Y = []
        '''
        for a in data_answer:
            #Getting indexes for each word in the story
            x = [word_index[word.lower()] for word in a]
            X.append(x)
        
        for a in data_question:
            #Getting indexes for each word in the story
            x = [word_index[word.lower()] for word in a]
            Xq.append(x)
        '''
        train_answers_seq = self.tokenizer.texts_to_sequences(data_answer)
        train_question_body_seq = self.tokenizer.texts_to_sequences(data_question)
            
        #Now we have to pad these sequences:
        return(pad_sequences(train_answers_seq,maxlen=max_story_len), 
               pad_sequences(train_question_body_seq, maxlen=max_question_len))    
   



In [0]:
nlpbefore_mdl = NLP_before_model(X_train['answer'], X_train['question_body'])   #Both train and test should be there

In [149]:
nlpbefore_mdl.tokenize_wordindex()

{'תורה': 1, 'bumped': 2, 'shatter': 3, 'supply': 4, 'religion': 5, 'integervalue': 6, '187': 7, 'lt2': 8, 'scrambleverse': 9, 'libnfs1': 10, 'devb': 11, 'parallax': 12, 'askoyne': 13, 'fallen': 14, '1page': 15, 'texture2d': 16, 'var_export': 17, 'beating': 18, 'opensession': 19, 'xmin_': 20, 'vncconfig': 21, 'bare': 22, 'socket': 23, 'lubing': 24, 'killprocess': 25, 'catalina': 26, 'confusingly': 27, 'sacrament': 28, 'comparatively': 29, 'depression': 30, 'unconditionally': 31, 'intruding': 32, 'superbest': 33, 'sensational': 34, 'cgit': 35, 'tooltip': 36, 'invadordamagable': 37, '107': 38, 'spinoffs': 39, 'ghs': 40, 'ngc7603': 41, 'aurl': 42, '444': 43, 'shadowrun': 44, '542': 45, 'unknot': 46, 'mousedoubleclick': 47, 'openstreetmap': 48, 'toefl': 49, 'birthdate': 50, 'sys_refcursor': 51, 'member_data': 52, 'שנוהגים': 53, 'error_log': 54, 'shui': 55, '0x00ff0000': 56, 'barrier': 57, 'element_type_name': 58, 'openhanded': 59, 'niether': 60, 'apx': 61, 'recap': 62, 'verbosely': 63, '145

In [151]:
#Vectorize trrain data
'''train_input_answer, train_input_question = nlpbefore_mdl.process_for_model()   #To vectorize only
train_input_answer'''

'train_input_answer, train_input_question = nlpbefore_mdl.process_for_model()   #To vectorize only\ntrain_input_answer'

In [150]:
#Vectorize trrain data
train_input_answer, train_input_question = nlpbefore_mdl.vectorize_stories(X_train['answer'], X_train['question_body'])   #To vectorize only
train_input_answer

array([[    0,     0,     0, ..., 15145, 27088,  9746],
       [    0,     0,     0, ..., 12112, 21324,  2672],
       [    0,     0,     0, ..., 30639, 24468, 34384],
       ...,
       [    0,     0,     0, ..., 18414,  3686,   263],
       [    0,     0,     0, ..., 36646, 37619,  7192],
       [    0,     0,     0, ...,  9512,  7321, 29836]], dtype=int32)

In [93]:
max_answers_seq_len = nlpbefore_mdl.max_answers_seq_len
max_question_body_len = nlpbefore_mdl.max_question_body_len
vocab_len = nlpbefore_mdl.vocab_len
print(max_answers_seq_len)
print(max_question_body_len)

2089
1832


In [94]:
#Vectorize test data
test_input_answer, test_input_question = nlpbefore_mdl.process_vectorize(X_test['answer'], X_test['question_body'])
test_input_answer

array([[    0,     0,     0, ..., 24006,  1194, 35883],
       [    0,     0,     0, ..., 33001, 34353, 21468],
       [    0,     0,     0, ..., 27029, 35765, 11365],
       ...,
       [    0,     0,     0, ..., 30214, 13350, 28052],
       [    0,     0,     0, ..., 23376,  3238, 31350],
       [    0,     0,     0, ..., 11817, 24232, 12757]], dtype=int32)

In [95]:
'''
max_answers_seq_len = nlpbefore_mdl.max_answers_seq_len
max_question_body_len = nlpbefore_mdl.max_question_body_len
vocab_len = nlpbefore_mdl.vocab_len
print(max_answers_seq_len)
print(max_question_body_len)
'''

'\nmax_answers_seq_len = nlpbefore_mdl.max_answers_seq_len\nmax_question_body_len = nlpbefore_mdl.max_question_body_len\nvocab_len = nlpbefore_mdl.vocab_len\nprint(max_answers_seq_len)\nprint(max_question_body_len)\n'

In [0]:
#vocablen = nlpbefore_mdl.process_vocab()
#print(vocablen)

In [97]:
print(max_answers_seq_len)
print(max_question_body_len)
print(vocab_len)

2089
1832
40839


In [0]:
from keras.optimizers import Adam
#from keras.losses import huber_loss

In [0]:
class NLP_model():
    max_answers_seq_len = 0
    max_question_body_len = 0
    vocab_len = 0
    
    model = 0
    history = 0
    a = 0
    
    
    def __init__(self, max_answers_seq_len, max_question_body_len, vocab_len):
        self.max_answers_seq_len = max_answers_seq_len
        self.max_question_body_len = max_question_body_len
        self.vocab_len = vocab_len
    
    def create_model(self):
        input_sequence = Input((self.max_answers_seq_len,)) #As we dont know batch size yet
        question = Input((self.max_question_body_len,))

        #Create input encoder M
        input_encoder_m = Sequential()
        input_encoder_m.add(Embedding(input_dim = self.vocab_len,output_dim = 64)) #From paper
        input_encoder_m.add(Dropout(0.3))

        #Create input encoder C:
        input_encoder_c = Sequential()
        input_encoder_c.add(Embedding(input_dim = self.vocab_len,output_dim = self.max_question_body_len)) #From paper
        input_encoder_c.add(Dropout(0.3))

        #Create question encoder:
        question_encoder = Sequential()
        question_encoder.add(Embedding(input_dim = self.vocab_len,output_dim = 64,input_length = self.max_question_body_len)) #From paper
        question_encoder.add(Dropout(0.3))

        input_encoded_m = input_encoder_m(input_sequence)
        input_encoded_c = input_encoder_c(input_sequence)
        question_encoded = question_encoder(question)

        match = dot([input_encoded_m,question_encoded], axes = (2,2))
        match = Activation('softmax')(match)

        response = add([match,input_encoded_c])
        response = Permute((2,1))(response) #Permute Layer: permutes dimensions of input

        answer = concatenate([response, question_encoded])

        # Reduce the answer tensor with a RNN (LSTM)
        answer = LSTM(32)(answer)

        #Regularization with dropout:
        answer = Dropout(0.5)(answer)
        #Output layer:
        answer = Dense(30)(answer) #Output shape: (Samples, Vocab_size) #Yes or no and all 0s

        #Now we need to output a probability distribution for the vocab, using softmax:
        answer = Activation('softmax')(answer)

        #Now we build the final model:
        self.model = Model([input_sequence,question], answer)

    def compile_model(self):
        #sgd = optimizers.SGD(lr=0.5, decay=1e-5, momentum=0.7, nesterov=True)  #0.01

        #self.model.compile(loss='poisson', optimizer=sgd, metrics = ['accuracy'])
        
        #model.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
        #model.compile(optimizer='rmsprop', loss = 'binary_crossentropy', metrics = ['accuracy'])
        #model.compile(optimizer='rmsprop', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
        # model.compile(optimizer='rmsprop', loss = 'categorical_crossentropy', metrics = ['accuracy'])
        #Categorical instead of binary cross entropy as because of the way we are training
        #we could actually see any of the words from the vocab as output
        #however, we should only see yes or no

        ## self.model.compile(optimizer='rmsprop', loss = 'categorical_crossentropy', metrics = ['accuracy']) #acc 6%
        #self.model.compile(optimizer='rmsprop', loss = 'binary_crossentropy', metrics = ['accuracy']) #accuracy around 75%
        self.model.compile(optimizer=Adam(lr=0.05), loss = 'binary_crossentropy', metrics = ['accuracy'])
        
    def summary_model(self):
        self.model.summary()
        
    def fit_model(self, input_labels, input_answer, input_question):
        #self.a = input_labels.astype(np.int64)
        #input_train == answeer  answert train == label // label left tot do
        #  history = model.fit([inputs_train,questions_train],a, batch_size = 32, epochs = 1000, validation_data = ([inputs_test,questions_test],answers_test))
        
        #print(type(self.a))
        #self.history = self.model.fit([input_answer,input_question], a, batch_size = 50, epochs = 1000)
        
        #self.history = self.model.fit([input_answer,input_question], input_labels, batch_size = 75, epochs = 1500)

        try:
          self.history = self.model.fit([input_answer,input_question], input_labels, batch_size = 75, epochs = 1000)
        except KeyboardInterrupt:
          self.save_model();
        #xxxxx
        
    def eval_model(self, answer, question):
        pred_results = self.model.predict(([answer,question]))
        
    def save_model(self):
        filename = 'google3qla.h5'
        self.model.save(filename)
        

In [101]:
nlpmodel = NLP_model(max_answers_seq_len, max_question_body_len, vocab_len)
nlpmodel.create_model()
nlpmodel.compile_model()
nlpmodel.summary_model()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 2089)         0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, 1832)         0                                            
__________________________________________________________________________________________________
sequential_7 (Sequential)       multiple             2613696     input_5[0][0]                    
__________________________________________________________________________________________________
sequential_9 (Sequential)       (None, 1832, 64)     2613696     input_6[0][0]                    
____________________________________________________________________________________________

In [104]:
nlpmodel.fit_model(y_train, train_input_answer, train_input_question)

Epoch 1/1000
4072/4072 [==============================] - 940s 231ms/step - loss: 0.7100 - acc: 0.7632
Epoch 2/1000
4072/4072 [==============================] - 923s 227ms/step - loss: 0.6888 - acc: 0.7632
Epoch 3/1000
4072/4072 [==============================] - 920s 226ms/step - loss: 0.6852 - acc: 0.7632
Epoch 4/1000
4072/4072 [==============================] - 925s 227ms/step - loss: 0.6840 - acc: 0.7632
Epoch 5/1000
3075/4072 [=====================>........] - ETA: 3:45 - loss: 0.6841 - acc: 0.7631

In [70]:
!ls

google3qla.h5  sample_data	      test.csv
kaggle.json    sample_submission.csv  train.csv.zip


In [0]:
#print(train_input_answer.shape)
nlpmodel.save_model()

In [0]:
from keras.models import load_model
ld_model = load_model('google3qla.h5')

In [99]:
print(train_input_answer.shape[1] - test_input_answer.shape[1])
print(train_input_question.shape[1] - test_input_question.shape[1])
print(train_input_answer.shape)
print(test_input_answer.shape)

1091
0
(4072, 2089)
(2007, 998)


In [41]:

y = np.zeros([2007, train_input_answer.shape[1] - test_input_answer.shape[1] ], dtype = int) #2007 is number of rows and 742 is column needed to match with train_answer
test_input_answer_1 = np.append(test_input_answer, y, axis=1)

print(test_input_answer_1.shape)
print(train_input_answer.shape)

y = np.zeros([2007, train_input_question.shape[1] - test_input_question.shape[1] ], dtype = int) #2007 is number of rows and 390 is column needed to match with train_question
test_input_question_1 = np.append(test_input_question, y, axis=1)

print(test_input_question_1.shape)
print(train_input_question.shape)

score = ld_model.evaluate([test_input_answer_1, test_input_question_1], y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

ValueError: ignored

In [0]:
print(y_test.shape)
print(test_input_answer.shape)
print(train_input_answer.shape)
print(type(test_input_answer))

In [0]:
print(test_input_question.shape)
print(train_input_question.shape)
print(type(test_input_question))

In [0]:
'''
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
'''

In [0]:
###   Rough Work  ###

In [110]:
!pip install helpers

In [0]:
import helpers
from sklearn import preprocessing
import numpy as np
X_test, y_test = helpers.get(subset="test")
helpers.
ohe = preprocessing.OneHotEncoder()
categories = np.array(list(set(X_test['workclass'].astype(str).values))).reshape(-1,1)


In [114]:
ohe.fit(categories)
#OneHotEncoder(categorical_features=None, categories=None,
 #      dtype=<class 'numpy.float64'>, handle_unknown='error',
  #     n_values=None, sparse=True)
categories
'''array([['Self-emp-inc'],
       ['Local-gov'],
       ['Private'],
       ['State-gov'],
       ['Never-worked'],
       ['Without-pay'],
       ['Federal-gov'],
       ['Self-emp-not-inc'],
       ['nan']], dtype='<U16')'''


NameError: ignored

In [0]:
ohe.transform(categories).todense()

In [0]:
'''
ylabels = ['question_asker_intent_understanding','question_body_critical','question_conversational','question_expect_short_answer',
'question_fact_seeking','question_has_commonly_accepted_answer','question_interestingness_others','question_interestingness_self',
'question_multi_intent','question_not_really_a_question','question_opinion_seeking','question_type_choice','question_type_compare',
'question_type_consequence','question_type_definition','question_type_entity','question_type_instructions','question_type_procedure',
'question_type_reason_explanation','question_type_spelling','question_well_written','answer_helpful','answer_level_of_information',
'answer_plausible','answer_relevance','answer_satisfaction','answer_type_instructions','answer_type_procedure','answer_type_reason_explanation',
'answer_well_written']

a = X_train[ylabels].astype(np.int64)
nlpmodel.fit_model(a, input_answer, input_question)
#nlpmodel.fit_model(X_train[ylabels])
'''

In [0]:
'''
def process_model_ready():
    tokenizer = Tokenizer(filters = [])
    tokenizer.fit_on_texts(vocab)

    train_question_body = []
    train_answers = []

    for a in X_train['answer']:
        train_answers.append(a)

    for a in X_train['question_body']:
        train_question_body.append(a)

    train_answers_seq = tokenizer.texts_to_sequences(train_answers)
    train_question_body_seq = tokenizer.texts_to_sequences(train_question_body)

    max_question_body_len = max([len(data) for data in train_question_body_seq])
    print(max_question_body_len)

    max_answers_seq_len = max([len(data) for data in train_answers_seq])
    print(max_answers_seq_len)

    input_answer = pad_sequences(train_answers_seq, maxlen=max_answers_seq_len)
    input_question = pad_sequences(train_question_body_seq, maxlen=max_question_body_len)
    
    return input_answer

input_answer = process_model_ready()
'''

In [0]:
'''
input_sequence = Input((max_answers_seq_len,)) #As we dont know batch size yet
question = Input((max_question_body_len,))

#Create input encoder M
input_encoder_m = Sequential()
input_encoder_m.add(Embedding(input_dim=vocab_len,output_dim = 64)) #From paper
input_encoder_m.add(Dropout(0.3))

#Create input encoder C:
input_encoder_c = Sequential()
input_encoder_c.add(Embedding(input_dim=vocab_len,output_dim = max_question_body_len)) #From paper
input_encoder_c.add(Dropout(0.3))

#Create question encoder:
question_encoder = Sequential()
question_encoder.add(Embedding(input_dim=vocab_len,output_dim = 64,input_length=max_question_body_len)) #From paper
question_encoder.add(Dropout(0.3))

input_encoded_m = input_encoder_m(input_sequence)
input_encoded_c = input_encoder_c(input_sequence)
question_encoded = question_encoder(question)

match = dot([input_encoded_m,question_encoded], axes = (2,2))
match = Activation('softmax')(match)

response = add([match,input_encoded_c])
response = Permute((2,1))(response) #Permute Layer: permutes dimensions of input

answer = concatenate([response, question_encoded])

# Reduce the answer tensor with a RNN (LSTM)
answer = LSTM(32)(answer)

#Regularization with dropout:
answer = Dropout(0.5)(answer)
#Output layer:
answer = Dense(2)(answer) #Output shape: (Samples, Vocab_size) #Yes or no and all 0s

#Now we need to output a probability distribution for the vocab, using softmax:
answer = Activation('softmax')(answer)

#Now we build the final model:
model = Model([input_sequence,question], answer)

from keras import optimizers
sgd = optimizers.SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)  #0.01

model.compile(loss='mean_squared_error', optimizer=sgd, metrics = ['accuracy'])
#model.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
#model.compile(optimizer='rmsprop', loss = 'binary_crossentropy', metrics = ['accuracy'])
#model.compile(optimizer='rmsprop', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
# model.compile(optimizer='rmsprop', loss = 'categorical_crossentropy', metrics = ['accuracy'])
#Categorical instead of binary cross entropy as because of the way we are training
#we could actually see any of the words from the vocab as output
#however, we should only see yes or no

model.summary()

#X_train['answer_well_written']
#X_train.columns
a = X_train[['answer_well_written','question_well_written']].astype(np.int64)
a

ylabels = ['question_asker_intent_understanding','question_body_critical','question_conversational','question_expect_short_answer',
'question_fact_seeking','question_has_commonly_accepted_answer','question_interestingness_others','question_interestingness_self',
'question_multi_intent','question_not_really_a_question','question_opinion_seeking','question_type_choice','question_type_compare',
'question_type_consequence','question_type_definition','question_type_entity','question_type_instructions','question_type_procedure',
'question_type_reason_explanation','question_type_spelling','question_well_written','answer_helpful','answer_level_of_information',
'answer_plausible','answer_relevance','answer_satisfaction','answer_type_instructions','answer_type_procedure','answer_type_reason_explanation',
'answer_well_written']

b = X_train[ylabels].astype(np.int64)
b

#input_train == answeer  answert train == label // label left tot do
#  history = model.fit([inputs_train,questions_train],a, batch_size = 32, epochs = 1000, validation_data = ([inputs_test,questions_test],answers_test))
  history = model.fit([input_answer,input_question],a, batch_size = 50, epochs = 1000)
  
'''

In [44]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
X = [['Male', 1], ['Female', 3], ['Female', 2]]
enc.fit(X)
OneHotEncoder(handle_unknown='ignore')
enc.categories_


[array(['Female', 'Male'], dtype=object), array([1, 2, 3], dtype=object)]

In [46]:
#[array(['Female', 'Male'], dtype=object), array([1, 2, 3], dtype=object)]
enc.transform([['Female', 1], ['Male', 4]]).toarray()
#array([[1., 0., 1., 0., 0.],
#       [0., 1., 0., 0., 0.]])


array([[1., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.]])

In [49]:
enc.inverse_transform([[0, 1, 1, 0, 0], [0, 0, 0, 1, 0], [0, 1, 0, 0, 0]])
#array([['Male', 1],
#       [None, 2]], dtype=object)


array([['Male', 1],
       [None, 2],
       ['Male', None]], dtype=object)

In [50]:
enc.get_feature_names(['gender', 'group'])
#array(['gender_Female', 'gender_Male', 'group_1', 'group_2', 'group_3'],
#  dtype=object)


array(['gender_Female', 'gender_Male', 'group_1', 'group_2', 'group_3'],
      dtype=object)

In [51]:
drop_enc = OneHotEncoder(drop='first').fit(X)
drop_enc.categories_
#[array(['Female', 'Male'], dtype=object), array([1, 2, 3], dtype=object)]


[array(['Female', 'Male'], dtype=object), array([1, 2, 3], dtype=object)]

In [52]:
drop_enc.transform([['Female', 1], ['Male', 2]]).toarray()
#array([[0., 0., 0.],
#       [1., 1., 0.]])

array([[0., 0., 0.],
       [1., 1., 0.]])

In [54]:
drop_enc.inverse_transform([[0, 0, 0],
       [1, 1, 0], [1,1,1]])

array([['Female', 1],
       ['Male', 2],
       ['Male', 2]], dtype=object)

In [0]:
from sklearn.preprocessing import OneHotEncoder
#enc = OneHotEncoder(handle_unknown='ignore')
X = [['Male', 1], ['Female', 3], ['Female', 2]]
#enc.fit(X)
#OneHotEncoder(handle_unknown='ignore')
#enc.categories_

In [65]:
drop_enc = OneHotEncoder(drop='first').fit(X)
#drop_enc = OneHotEncoder().fit(X)
drop_enc.categories_

[array(['Female', 'Male'], dtype=object), array([1, 2, 3], dtype=object)]

In [66]:
drop_enc.transform([['Female', 1], ['Male', 2]]).toarray()

array([[0., 0., 0.],
       [1., 1., 0.]])

In [58]:
drop_enc.inverse_transform([[0, 0, 0],
       [1, 1, 0], [1,1,1]])

array([['Female', 1],
       ['Male', 2],
       ['Male', 2]], dtype=object)

In [67]:
X = y_train
drop_enc = OneHotEncoder(drop='first').fit(X)
#drop_enc = OneHotEncoder().fit(X)
drop_enc.categories_

[array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1]),
 array([0, 1])]

In [74]:
#y_train.iloc[[1]]
drop_enc.transform([[1,	0,	0,	1,	0,	1,	0,	0,	0,	0,	0,	0,	0,	0,	0,	0,	1,	0,	1,	0,	0,	1,	0,	1,	1,	0,	1,	0,	0,	1]]).toarray()

array([[1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        1., 0., 1., 0., 1., 0., 1., 1., 0., 1., 0., 0., 1.]])

In [75]:
drop_enc.inverse_transform([[1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1]])

array([[1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1,
        0, 1, 1, 0, 1, 0, 0, 1]])

In [0]:
from sklearn.preprocessing import OneHotEncoder

X = [['Male', 1], ['Female', 3], ['Female', 2]]
ohe = OneHotEncoder(categories='auto')
feature_arr = ohe.fit_transform(X).toarray()
feature_labels = ohe.categories_

In [77]:
ohe.fit_transform(X).toarray()

array([[0., 1., 1., 0., 0.],
       [1., 0., 0., 0., 1.],
       [1., 0., 0., 1., 0.]])